# LCPB 20-21 exercise 4

### Saverio Monaco

### Gerardo Carmona

### Hilario Capettini

In [ ]:
#!pip install plotly #to install the interactive plots library (not mandatory)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

from sklearn.manifold import TSNE
from itertools import cycle, islice
import matplotlib.patheffects as PathEffects

try:
    import plotly.express as px
    import plotly.graph_objects as go
except:
    print('Interactive plots not supported')
    
seed = 31415

In [ ]:
def cl_scatter3d_static(xdata,ydata,title = False):
    colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                     '#f781bf', '#a65628', '#984ea3',
                                     '#999999', '#e41a1c', '#dede00']),
                                    int(max(ydata) + 1))))
    fig, (ax1) = plt.subplots(figsize=(10, 10))
    ax1 = plt.axes(projection='3d')
    ax1.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax1.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax1.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax1.set_yticklabels([])
    ax1.set_xticklabels([])
    ax1.set_zticklabels([])
    #ax1.view_init(view1, view2)
    if title:
        ax1.set_title(title)
    ax1.scatter3D(xdata[:,0], xdata[:,1], xdata[:,2],color=colors[ydata]);
    
def cl_scatter3d_interactive(xdata,ydata,size=2,title=False):
    colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                     '#f781bf', '#a65628', '#984ea3',
                                     '#999999', '#e41a1c', '#dede00']),
                                     int(max(ydata) + 1))))
    fig = go.Figure(data=[go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                       mode='markers', marker=dict(color=colors[ydata],size=size))])
    if title:
        fig.update_layout(title=title)
    fig.show()

# more general function, it checks if the interactive plots library is installed, if so it will show
# the interactive plot, if not it will use matplotlib library
def cl_scatter3d(xdata,ydata,size=2,title=False):
    if('plotly' in sys.modules):
        cl_scatter3d_interactive(xdata,ydata,size,title)
    else:
        cl_scatter3d_static(xdata,ydata,title)
        

# PART 1

## 1 Read data files as “data_t-SNE_310101_d5_R100_e1_N800.dat”

In [ ]:
data = np.loadtxt('./DATA/data_t-SNE_310101_d5_R100_e1_N800.dat',delimiter='\t')
data_size = np.shape(data)[0]


In [ ]:
data_y = []
for i in range(int(data_size/10)):
    data_y.append(0)

for i in range(int(data_size*3/10)):
    data_y.append(1)

for i in range(int(data_size*6/10)):
    data_y.append(2)

In [ ]:
cl_scatter3d(data,data_y)

## 2 Apply t-SNE with 4 different perplexities to the data.

class sklearn.manifold.TSNE(n_components=2, *, perplexity=30.0, early_exaggeration=12.0, learning_rate=200.0, n_iter=1000, n_iter_without_progress=300, min_grad_norm=1e-07, metric='euclidean', init='random', verbose=0, random_state=None, method='barnes_hut', angle=0.5, n_jobs=None, square_distances='legacy')

In [ ]:
def memo(f):
    args = []
    results = []
    def helper(arg1,**kwargs):
        t = tuple([arg1]+[arg for arg in kwargs.values()])
        if t not in args:
            args.append(t)
            results.append(f(arg1,**kwargs))
        return results[args.index(t)]
    return helper

#@memo # using a memoized version of the method so as not to waste time recalculating
def tsne_2(x,perplexity = 5,learning_rate=200.0,metric='euclidean',init='random',random_state=seed):
    X = TSNE(n_components=2,
             perplexity=perplexity,
             learning_rate=learning_rate,
             metric=metric,
             init=init,
             random_state=random_state).fit_transform(x)
    X.shape
    return X


In [ ]:
colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                     '#f781bf', '#a65628', '#984ea3',
                                     '#999999', '#e41a1c', '#dede00']),
                                    int(max(data_y) + 1))))


def grid2plot(data,method,**kwargs):
    '''Takes up to two grid parameters and plots in a convenient way'''
    (rowp, rows), (colp,cols) = kwargs.items()
    fig, plotties = plt.subplots(nrows=len(rows), ncols=len(cols), figsize=(20, 10))
    for i, r in enumerate(rows):
        for j, c in enumerate(cols):
            x = method(data, **{rowp: r, colp: c})
            plotties[i,j].scatter(x[:,[0]],x[:,[1]],color=colors[data_y])
            plotties[i,j].set_title(f'{rowp} = {r}, {colp} = {c}')
            plotties[i,j].set_yticklabels([])
            plotties[i,j].set_xticklabels([])
    plt.show()
                       

In [ ]:
perplexities = [5,30,50,100]
inits = ['random', 'pca']

grid2plot(data, tsne_2, init = inits, perplexity = perplexities)


The plots on the first row were done initializing the t-sne algorithm randomly while the second case was done with pca. We can see that after the initialization with pca the result doesn't change as we modify the perplexity parameter. This might be because the pca algorithm already picked two variables which represent most of the data variability so t-sne cannot go further.

## 3. From NB15 extract the part concerning the DBSCAN algorithm for clustering and embed it in the notebook to analyze the clustering in d=5 dimensions to generate predicted labels, then plot the results. Something like the next figure should arise. It includes a grid with several values of “eps” and “minPts”.
## To understand a good range for the cutoffs “eps” in DBSCAN, sort all minimum distances to first neighbors and plot them.


In [ ]:
from collections import OrderedDict
cpalette = ['k','r','g','b','y']#["#1CE6FF", "#FF34FF", "#FF4A46","#008941", "#006FA6", "#A30059", "#0000A6", "#63FFAC","#B79762", "#004D43", "#8FB0FF", "#997D87","#5A0007", "#809693","#1B4400", "#4FC601", "#3B5DFF", "#4A3B53","#886F4C","#34362D", "#B4A8BD", "#00A6AA", "#452C2C","#636375", "#A3C8C9", "#FF913F", "#938A81","#575329", "#00FECF", "#B05B6F"]
def clustering(y):
    # Finds position of labels and returns a dictionary of cluster labels to data indices.
    yu = np.sort(np.unique(y))
    clustering = OrderedDict()
    for ye in yu:
        clustering[ye] = np.where(y == ye)[0]
    return clustering

def entropy(c, n_sample):
    # Measures the entropy of a cluster
    h = 0.
    for kc in c.keys():
        p=len(c[kc])/n_sample
        h+=p*np.log(p)
    h*=-1.
    return h

def NMI(y_true, y_pred):
    """ Computes normalized mutual information: where y_true and y_pred are both clustering assignments
    """
    w = clustering(y_true)
    c = clustering(y_pred)
    n_sample = len(y_true)

    Iwc = 0.
    for kw in w.keys():
        for kc in c.keys():
            w_intersect_c=len(set(w[kw]).intersection(set(c[kc])))
            if w_intersect_c > 0:
                Iwc += w_intersect_c*np.log(n_sample*w_intersect_c/(len(w[kw])*len(c[kc])))
    Iwc/=n_sample
    Hc = entropy(c,n_sample)
    Hw = entropy(w,n_sample)

    return 2*Iwc/(Hc+Hw)


from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

def plotting_ax(X, y, ax):
    # plotting function
    for i, yu in enumerate(np.unique(y)):
        pos = (y == yu)
        ax.scatter(X[pos,0], X[pos,1],c=cpalette[i%len(cpalette)],s=16)

def DBSCAN_grid(input_x,input_y,eps_range,min_sample_range):
    # DBSCAN has a few parameters, let's sweep over a few parameters and see what happens
    np.random.seed(0)
    n_true_center=len(np.unique(input_y))
    X = StandardScaler().fit_transform(input_x)

    eps_range = eps_range 
    min_sample_range = min_sample_range 
    fig, ax = plt.subplots(len(eps_range),len(min_sample_range),figsize=(15,10))
    for i, eps in enumerate(eps_range):
        for j, min_samples in enumerate(min_sample_range):
            model = DBSCAN(eps=eps, min_samples=min_samples)
            model.fit(input_x)
            _y = model.labels_
            plotting_ax(input_x,_y,ax[i,j])
            nmi=NMI(input_y, _y)
            ax[i,j].set_title('eps=%.2f, minPts=%i, nmi=%.2f'%(eps,min_samples,nmi),fontweight='bold')
        
            ax[i,j].set_yticklabels([])
            ax[i,j].set_xticklabels([])
            # Because the next figure is too dense of information, only the most left axes will show the values
            # and only the bottom figures will have the x axes
            #if(not i): # if i is zero (first row) do not display the x axis
            #    ax[i,j].set_xticks([])
            #if(j): # if j is not zero (not the first column) do not display the y axis
            #    ax[i,j].set_yticks([])
            
    plt.tight_layout(h_pad=0.5)
    fig.subplots_adjust(hspace=.15)
    plt.show()    


def DBSCAN_grid_accuracy(input_x,input_y,eps_range,min_sample_range,metric='euclidean'):
    # DBSCAN has a few parameters, let's sweep over a few parameters and see what happens

    eps_range = eps_range 
    min_sample_range = min_sample_range
    print('eps, minPts, nmi')
    for i, eps in enumerate(eps_range):
        for j, min_samples in enumerate(min_sample_range):
            model = DBSCAN(eps=eps, min_samples=min_samples,metric=metric)
            model.fit(input_x)
            _y = model.labels_
            nmi=NMI(input_y, _y)
            print('%.2f, %2i, %.2f'%(eps,min_samples,nmi))

In [ ]:
DBSCAN_grid(data,data_y,[8400, 16800],[3,5,10,20])


# PART 2

In [ ]:
def clustercentroids(tsnex,y):
    numclusters = int(max(y)+1)
    numx = np.shape(tsnex)[0]
    centroids = []
    for j in range(numclusters):
        den = 0
        xcord = 0
        ycord = 0
        for i in range(numx):
            if y[i] == j:
                den += 1
                xcord += tsnex[i,[0]]
                ycord += tsnex[i,[1]]
        centroids.append([xcord/den,ycord/den])
    return centroids

In [ ]:
x = np.loadtxt('DATA/x_M5_N400.csv', delimiter=' ')
y = np.loadtxt('DATA/y_M5_N400.csv')


In [ ]:
for _ in range(4):
    print(x[_].astype(int),y[_].astype(int))
colors = np.array(list(islice(cycle(['black','red','green',
                                     'blue','y', '#984ea3',
                                     '#999999', '#e41a1c', '#dede00']),
                                     int(max(y) + 1))))

perp = 25
fig, (ax1) = plt.subplots(figsize=(5, 5))
tsnex = tsne_2(x,perp)
textcord = clustercentroids(tsnex,y)
for i in range(0,max(y.astype(int))+1):
    txt = ax1.text(textcord[i][0], textcord[i][1], i, fontsize=25)
    txt.set_path_effects([PathEffects.withStroke(linewidth=5, foreground='w')])
ax1.scatter(tsnex[:,[0]],tsnex[:,[1]],color=colors[y.astype(int)])
ax1.set_title('Perplexity='+str(perp))
plt.show()

In [ ]:
# the function that computes the distance between two sequences
# here the distance is defined as the number of different bits
# the function computes that
def twoseq_distance(x1,x2):
    xlen = np.shape(x1)[0]
    dist = 0
    for i in range(xlen):
        if x1[i]!=x2[i]:
            dist = dist + 1
    return dist


In [ ]:
# Let's check if it is correct by printing two sequences
# you can verify it
print(x[0].astype(int))
print(x[1].astype(int))
print('\nThe number of different bits of the two sequences should be',twoseq_distance(x[0],x[1]))


In [ ]:
# Here we compute what is called the minimum distances to first neighbors
# What i did is computing for each sequence, the distance from every other point (even from the ones 
# the same label) and took the lowest one
# Then i sorted the minimum distances

# Since we need to find the minima, I initialized the minima array to high values
xmin = np.zeros(np.shape(x)[0])+1000

# For every sequence in our dataset...
for i in range(np.shape(x)[0]):
    # We calculate the distance to every other point that is not itself...
    for j in range(np.shape(x)[0]):
        if i!=j:
            xixj = twoseq_distance(x[i],x[j])
            # if the distance calculated between i and j is lower than the one initialized or previously
            # calculated then we update our minima array.
            if xixj <= xmin[i]:
                xmin[i] = xixj


In [ ]:
plt.plot(np.sort(xmin))
plt.xlabel('rank')
plt.ylabel('distance to 1st neighbor')
plt.grid(b=True, which='both', color='0.65', linestyle='-')

In [ ]:
def meandistance_twogroups(x,y,y1,y2):
    N = 0
    distances = 0
    for i in range(np.shape(x)[0]):
        if y[i]==y1:
            for j in range(np.shape(x)[0]):
                if y[j]==y2:
                    N = N + 1
                    distances += twoseq_distance(x[i],x[j])
    return distances/N

In [ ]:
labels      = []
distances   = []
bar_colors  = []
edge_colors = []

for i in range(5):
    for j in range(5):
        if i>j:
            if i == 4 and j == 1 or i ==1 and j ==4:
                bar_colors.append('yellow')
                edge_colors.append('r')
            else:
                bar_colors.append('lightgray')
                edge_colors.append('lightgray')
            labels.append(str(i)+'-'+str(j))
            distances.append(meandistance_twogroups(x,y,i,j))

In [ ]:
list1, list2, bcolors, ecolors = zip(*sorted(zip(distances, labels, bar_colors, edge_colors)))

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(list2,list1,color=bcolors,hatch='//',edgecolor=ecolors)
plt.show()

In [ ]:
DBSCAN_grid_accuracy(x,y,[2,3,5,6],[3,5,10,20],metric='l1')

From the previous table it can be seen that DBSCAn algorithm for clustering is failing while trying to perform the clustering even when the appropiate characteristic distances and metric are used. 

In [ ]:
sne_x = tsne_2(x, perplexity=20)

In [ ]:
eps_range = [2,3,5,6]
min_sample_range = [3,5,10,20]
DBSCAN_grid(sne_x,y,eps_range,min_sample_range)